This architecture will be slightly different from language translation seq2seq. In translation model, the decoder output of last timestep will be feed into the model as decoder input for this timestep. I think people calls it inference model...Anyway we are not going to use that structure.

We will be used past t days data to predict future 39 days, in alignment with the submission sample data.
In our case the encoder will be feed in (n_stores, t_timesteps, m_features). m_features will be the day info, reservation info, store info, weather info we could also include the one_hot_encoding of the store_id,,,or maybe not..we will see,,
The decoder would be feed in with the hidden state of encoder and also decoder_input(which are the features of dates info, weather info )

encoder input data:
one sample(store) would be like

| time| v1 | v2 | v3 | v4 | v5 |
| --- | --- | --- | --- | --- | ---|
|t=1|rain|genre1|visitors 11| reservation 5|
|t=2|sunny|genre1|visitors 20| reservation 5|
|t=t|xxx|xxx|xxxx|xxxxx|

decoder input data:

| time| v1 | v2 | v3 |
| --- | --- | --- | --- |
|t=t+1|genre1|region1|date info|
|t=t+2|genre1|region1|date info|


decoder target data:
the same sample(store)

| time| target | 
| --- | ------ |
|t=t+1|visitors|
|t=t+39|visitors|

In [39]:
import pandas as pd
import numpy as np
import random
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_seq_items = 2000

In [40]:
full = pd.read_csv('master_final.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

# transform submission test data
samp_sub = sample_submission.copy()
samp_sub['air_store_id'] = samp_sub['id'].apply(lambda x: str(x)[:-11])
samp_sub['calendar_date'] = samp_sub['id'].apply(lambda x: str(x)[-10:])

In [41]:
full['calendar_date'] = pd.to_datetime(full['calendar_date'])
samp_sub['calendar_date'] = pd.to_datetime(samp_sub['calendar_date']).dt.date

In [42]:
full.head(2)

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,reserve_visitors_air,hpg_store_id,reserve_visitors_hpg,visitors,day,month,year,dayofweek,dayofyear,earlist_open_date,closed_flag,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area
0,air_00a91d42b08b08d9,2016-01-01,Friday,1,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,hpg_fbe603376b5980fc,NaN,NaN,1,1,2016,4,1,2016-07-01,2,26.08,23.282404,24.763514,7.5,12.1,3.6,0.0,9.0,2.6,45.0,0.0,0.0,one,0,8.406347,13.85957,14.56454
1,air_00a91d42b08b08d9,2016-01-02,Saturday,1,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN,hpg_fbe603376b5980fc,NaN,NaN,2,1,2016,5,2,2016-07-01,2,26.08,23.282404,24.763514,7.3,13.4,2.6,0.0,7.5,1.9,51.0,0.0,0.0,one,0,8.406347,13.85957,14.56454


In [43]:
full.tail(2)

,air_store_id,calendar_date,day_of_week,holiday_flg,air_genre_name,air_area_name,latitude,longitude,reserve_visitors_air,hpg_store_id,reserve_visitors_hpg,visitors,day,month,year,dayofweek,dayofyear,earlist_open_date,closed_flag,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area
428591,air_fff68b929994bfbd,2017-05-30,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,NaN,NaN,NaN,NaN,30,5,2017,1,150,2016-07-01,1,5.0,13.060498,15.125000,24.0,31.5,17.6,0.0,11.9,1.0,70.000002,0.0,0.0,four/five,333,2.629956,10.344121,4.274091
428592,air_fff68b929994bfbd,2017-05-31,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,NaN,NaN,NaN,NaN,31,5,2017,2,151,2016-07-01,1,4.0,10.864111,14.217391,24.0,29.6,20.5,0.0,3.4,1.2,76.000002,0.0,0.0,four/five,334,3.774917,12.448389,4.450789


In [44]:
full.columns

Index(['air_store_id', 'calendar_date', 'day_of_week', 'holiday_flg',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'reserve_visitors_air', 'hpg_store_id', 'reserve_visitors_hpg',
       'visitors', 'day', 'month', 'year', 'dayofweek', 'dayofyear',
       'earlist_open_date', 'closed_flag', 'lookback_visitors_avg_store',
       'lookback_visitors_avg_genre', 'lookback_visitors_avg_area',
       'avg_temperature', 'high_temperature', 'low_temperature',
       'precipitation', 'hours_sunlight', 'avg_wind_speed', 'avg_humidity',
       'day_after_holiday_flg', 'day_before_holiday_flg', 'week_of_month',
       'open_days', 'lookback_visitors_std_store',
       'lookback_visitors_std_genre', 'lookback_visitors_std_area'],
      dtype='object')

In [45]:
print('full table has',full.air_store_id.nunique(),'stores' )
print('full table calendar period', full.calendar_date.min(), '--', full.calendar_date.max())
print('Sample submission file has',samp_sub.air_store_id.nunique(),'stores' )
print('Sample submission prediction period is', samp_sub.calendar_date.min(), '--', samp_sub.calendar_date.max())
print('On total we need to predict %s days.'%(int((samp_sub.calendar_date.max()-samp_sub.calendar_date.min()).days)+1))
print('Therefore the decoder output timestep would be 39 days, but we can arbitrarily define the encoder input timestep')

full table has 829 stores
full table calendar period 2016-01-01 00:00:00 -- 2017-05-31 00:00:00
Sample submission file has 821 stores
Sample submission prediction period is 2017-04-23 -- 2017-05-31
On total we need to predict 39 days.
Therefore the decoder output timestep would be 39 days, but we can arbitrarily define the encoder input timestep


In [46]:
pd.to_datetime('2016-01-01') - pd.to_datetime('2017-04-22') # 478 days that we have visitors info

Timedelta('-477 days +00:00:00')

# Define train_validation split date

A walk forward split for train and validation dataset, details can be found in the following links
[https://github.com/JEddy92/TimeSeries_Seq2Seq/blob/master/notebooks/TS_Seq2Seq_Intro.ipynb]

1. Train encoding period (400 days)
2. Train decoding period (train targets, 39 days)
3. Validation encoding period (400 days)
4. Validation decoding period (validation targets, 39 days)

In [47]:
from datetime import timedelta
import datetime

In [48]:
LOOKBACK_DAYS = 400
PRED_DAYS = 39

In [49]:
# create a function that will give us the time period for training model
# in default we would use 400 days as train encoding, because we have in total 477 days that have visitors info, 
# and we need to shift to accomondate the validation period.
def calculate_date_split(data_start_date ='2016-01-01', validation_end_date= '2017-04-22', t = 400, pred_days=39):
    val_pred_end = pd.to_datetime(validation_end_date)
    val_pred_start = pd.to_datetime(validation_end_date) - timedelta(days= pred_days - 1)
    
    val_enc_end = val_pred_start - timedelta(days = 1)
    val_enc_start = val_enc_end - timedelta(days = t - 1)
    
    train_pred_end = val_enc_end
    train_pred_start = train_pred_end - timedelta(days= pred_days - 1)
    
    train_enc_end = train_pred_start - timedelta(days = 1)
    train_enc_start = train_enc_end - timedelta(days = t - 1)
    
    if train_enc_start < pd.to_datetime(data_start_date):
        print('the tranining encoding period out of date')
    
    test_pred_start = val_pred_end + timedelta(days = 1) 
    test_pred_end = test_pred_start + timedelta(days = pred_days -1)
    
    test_enc_end = test_pred_start - timedelta(days = 1)
    test_enc_start = test_enc_end - timedelta(days = t - 1)

    return test_enc_start, test_enc_end, test_pred_start, test_pred_end,\
           train_enc_start, train_enc_end, train_pred_start,train_pred_end, \
           val_enc_start, val_enc_end, val_pred_start, val_pred_end

In [50]:
test_enc_start, test_enc_end, test_pred_start, test_pred_end, \
train_enc_start, train_enc_end, train_pred_start,train_pred_end, \
val_enc_start, val_enc_end, val_pred_start, val_pred_end = calculate_date_split(t = LOOKBACK_DAYS, pred_days = PRED_DAYS)

In [51]:
print('Train encoding:', train_enc_start, '-', train_enc_end)
print('Train prediction:', train_pred_start, '-', train_pred_end, '\n')
print('Val encoding:', val_enc_start, '-', val_enc_end)
print('Val prediction:', val_pred_start, '-', val_pred_end,'\n')
print('Test encoding:', test_enc_start, '-', test_enc_end)
print('Test prediction:', test_pred_start, '-', test_pred_end)

print('\nEncoding interval:', (train_pred_start - train_enc_start).days)
print('Prediction interval:', (train_pred_end - train_enc_end).days)

Train encoding: 2016-01-01 00:00:00 - 2017-02-03 00:00:00
Train prediction: 2017-02-04 00:00:00 - 2017-03-14 00:00:00 

Val encoding: 2016-02-09 00:00:00 - 2017-03-14 00:00:00
Val prediction: 2017-03-15 00:00:00 - 2017-04-22 00:00:00 

Test encoding: 2016-03-19 00:00:00 - 2017-04-22 00:00:00
Test prediction: 2017-04-23 00:00:00 - 2017-05-31 00:00:00

Encoding interval: 400
Prediction interval: 39


# Feature Selection And Normalization
reference from[https://www.kaggle.com/ievgenvp/lstm-encoder-decoder-via-keras-lb-0-5]

1. nominal feature label/or ??onehot encoding,,,use numeric label encoder...see what we would got..
2. minmax scaling selected features that would go into the model

### step1

In [52]:
full.loc[full['week_of_month'] == 'one','week_of_month'] = 1
full.loc[full['week_of_month'] == 'two','week_of_month'] = 2
full.loc[full['week_of_month'] == 'three','week_of_month'] = 3
full.loc[full['week_of_month'] == 'four/five','week_of_month'] = 4

In [53]:
# one hot encoding get dummy variables
full = pd.concat([full,pd.get_dummies(full['air_genre_name'], prefix = 'genre').astype(float)], axis = 1)
full = pd.concat([full,pd.get_dummies(full['air_area_name'], prefix = 'area').astype(float)], axis = 1)

In [44]:
# label encoder
#from sklearn.preprocessing import LabelEncoder
# Genre name
#le_genre = LabelEncoder()
#le_genre.fit(full['air_genre_name'])
#full['air_genre_name_ix'] = le_genre.transform(full['air_genre_name'])

# Area name
#le_area = LabelEncoder()
#le_area.fit(full['air_area_name'])
#full['air_area_name_ix'] = le_area.transform(full['air_area_name'])

In [28]:
full.iloc[:,2:].columns.get_loc("lookback_visitors_std_area")

33

In [26]:
full.dtypes

air_store_id                                                  object
calendar_date                                         datetime64[ns]
day_of_week                                                   object
holiday_flg                                                    int64
air_genre_name                                                object
air_area_name                                                 object
latitude                                                     float64
longitude                                                    float64
reserve_visitors_air                                         float64
hpg_store_id                                                  object
reserve_visitors_hpg                                         float64
visitors                                                     float64
day                                                            int64
month                                                          int64
year                              

In [32]:
full.columns 

Index(['air_store_id', 'calendar_date', 'day_of_week', 'holiday_flg',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'reserve_visitors_air', 'hpg_store_id', 'reserve_visitors_hpg',
       'visitors', 'day', 'month', 'year', 'dayofweek', 'dayofyear',
       'earlist_open_date', 'closed_flag', 'lookback_visitors_avg_store',
       'lookback_visitors_avg_genre', 'lookback_visitors_avg_area',
       'avg_temperature', 'high_temperature', 'low_temperature',
       'precipitation', 'hours_sunlight', 'avg_wind_speed', 'avg_humidity',
       'day_after_holiday_flg', 'day_before_holiday_flg', 'week_of_month',
       'open_days', 'lookback_visitors_std_store',
       'lookback_visitors_std_genre', 'lookback_visitors_std_area',
       'genre_Asian', 'genre_Bar/Cocktail', 'genre_Cafe/Sweets',
       'genre_Creative cuisine', 'genre_Dining bar',
       'genre_International cuisine', 'genre_Italian/French', 'genre_Izakaya',
       'genre_Japanese food', 'genre_Karaoke/Part

In [64]:
# select the features that you want
# we keep air_store_id and claendar_date columns to transform input data shape later

# we purposefully put unstationary features in the end of the list
# by stationary I mean genre,area,day of year etc.. those features are certainly known before prediction.
# because we will use stationary features as decoder input later

selected_cols = ['air_store_id', 'calendar_date', 'visitors', 'holiday_flg',
       'latitude', 'longitude',
       'day', 'month', 'year', 'dayofweek', 'dayofyear',
       
       'lookback_visitors_avg_store',
       'lookback_visitors_avg_genre', 'lookback_visitors_avg_area',
       'avg_temperature', 'high_temperature', 'low_temperature',
       'precipitation', 'hours_sunlight', 'avg_wind_speed', 'avg_humidity',
       'day_after_holiday_flg', 'day_before_holiday_flg', 'week_of_month',
       'open_days', 'lookback_visitors_std_store',
       'lookback_visitors_std_genre', 'lookback_visitors_std_area',
                 
       'genre_Asian', 'genre_Bar/Cocktail', 'genre_Cafe/Sweets',
       'genre_Creative cuisine', 'genre_Dining bar',
       'genre_International cuisine', 'genre_Italian/French', 'genre_Izakaya',
       'genre_Japanese food', 'genre_Karaoke/Party',
       'genre_Okonomiyaki/Monja/Teppanyaki', 'genre_Other',
       'genre_Western food', 'genre_Yakiniku/Korean food',
       'area_Fukuoka-ken Fukuoka-shi Daimyō',
       'area_Fukuoka-ken Fukuoka-shi Hakata Ekimae',
       'area_Fukuoka-ken Fukuoka-shi Imaizumi',
       'area_Fukuoka-ken Fukuoka-shi Momochi',
       'area_Fukuoka-ken Fukuoka-shi Shiobaru',
       'area_Fukuoka-ken Fukuoka-shi Takatori',
       'area_Fukuoka-ken Fukuoka-shi Tenjin',
       'area_Fukuoka-ken Fukuoka-shi Torikai',
       'area_Fukuoka-ken Itoshima-shi Maebarunishi',
       'area_Fukuoka-ken Kitakyūshū-shi Konyamachi',
       'area_Fukuoka-ken Kitakyūshū-shi None',
       'area_Fukuoka-ken Kitakyūshū-shi Ōtemachi',
       'area_Fukuoka-ken Kurume-shi Jōnanmachi',
       'area_Fukuoka-ken Yame-shi Motomachi',
       'area_Hiroshima-ken Fukuyama-shi Higashisakuramachi',
       'area_Hiroshima-ken Hiroshima-shi Kokutaijimachi',
       'area_Hiroshima-ken Hiroshima-shi Nagarekawachō',
       'area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi',
       'area_Hokkaidō Asahikawa-shi 6 Jōdōri',
       'area_Hokkaidō Katō-gun Motomachi',
       'area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō',
       'area_Hokkaidō Sapporo-shi Kita 24 Jōnishi',
       'area_Hokkaidō Sapporo-shi Kotoni 2 Jō',
       'area_Hokkaidō Sapporo-shi Minami 3 Jōnishi',
       'area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō',
       'area_Hyōgo-ken Himeji-shi Yasuda',
       'area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike',
       'area_Hyōgo-ken Kōbe-shi Kumoidōri',
       'area_Hyōgo-ken Kōbe-shi Motomachidōri',
       'area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi',
       'area_Hyōgo-ken Nishinomiya-shi Rokutanjichō',
       'area_Hyōgo-ken Takarazuka-shi Tōyōchō',
       'area_Miyagi-ken Sendai-shi Kamisugi',
       'area_Niigata-ken Kashiwazaki-shi Chūōchō',
       'area_Niigata-ken Nagaoka-shi Ōtedōri',
       'area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō',
       'area_Niigata-ken Niigata-shi Teraohigashi',
       'area_Shizuoka-ken Hamamatsu-shi Motoshirochō',
       'area_Shizuoka-ken Numazu-shi Miyukichō',
       'area_Shizuoka-ken Shizuoka-shi Ōtemachi',
       'area_Tōkyō-to Adachi-ku Chūōhonchō', 'area_Tōkyō-to Bunkyō-ku Kasuga',
       'area_Tōkyō-to Bunkyō-ku Sendagi',
       'area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō',
       'area_Tōkyō-to Chiyoda-ku Kudanminami', 'area_Tōkyō-to Chūō-ku Ginza',
       'area_Tōkyō-to Chūō-ku Tsukiji', 'area_Tōkyō-to Edogawa-ku Chūō',
       'area_Tōkyō-to Fuchū-shi Miyanishichō',
       'area_Tōkyō-to Itabashi-ku Itabashi',
       'area_Tōkyō-to Katsushika-ku Tateishi',
       'area_Tōkyō-to Kita-ku Ōjihonchō', 'area_Tōkyō-to Koganei-shi Honchō',
       'area_Tōkyō-to Kōtō-ku Kameido', 'area_Tōkyō-to Kōtō-ku Tomioka',
       'area_Tōkyō-to Kōtō-ku Tōyō', 'area_Tōkyō-to Machida-shi Morino',
       'area_Tōkyō-to Meguro-ku Jiyūgaoka',
       'area_Tōkyō-to Meguro-ku Kamimeguro', 'area_Tōkyō-to Meguro-ku Takaban',
       'area_Tōkyō-to Minato-ku Akasaka', 'area_Tōkyō-to Minato-ku Roppongi',
       'area_Tōkyō-to Minato-ku Shibakōen',
       'area_Tōkyō-to Musashino-shi Kichijōji Honchō',
       'area_Tōkyō-to Musashino-shi Midorichō',
       'area_Tōkyō-to Nakano-ku Nakano',
       'area_Tōkyō-to Nerima-ku Toyotamakita',
       'area_Tōkyō-to Setagaya-ku Kitazawa',
       'area_Tōkyō-to Setagaya-ku Okusawa',
       'area_Tōkyō-to Setagaya-ku Sangenjaya',
       'area_Tōkyō-to Setagaya-ku Setagaya',
       'area_Tōkyō-to Shibuya-ku Dōgenzaka',
       'area_Tōkyō-to Shibuya-ku Higashi', 'area_Tōkyō-to Shibuya-ku Jingūmae',
       'area_Tōkyō-to Shibuya-ku Shibuya',
       'area_Tōkyō-to Shinagawa-ku Higashigotanda',
       'area_Tōkyō-to Shinagawa-ku Hiromachi',
       'area_Tōkyō-to Shinjuku-ku Kabukichō',
       'area_Tōkyō-to Shinjuku-ku Nishishinjuku',
       'area_Tōkyō-to Suginami-ku Asagayaminami',
       'area_Tōkyō-to Tachikawa-shi Izumichō',
       'area_Tōkyō-to Taitō-ku Asakusa', 'area_Tōkyō-to Taitō-ku Higashiueno',
       'area_Tōkyō-to Taitō-ku Kuramae',
       'area_Tōkyō-to Toshima-ku Higashiikebukuro',
       'area_Tōkyō-to Toshima-ku Mejiro',
       'area_Tōkyō-to Toshima-ku Minamiikebukuro',
       'area_Tōkyō-to Toshima-ku Nishiikebukuro',
       'area_Tōkyō-to Toshima-ku Sugamo', 'area_Tōkyō-to Ōta-ku Kamata',
       'area_Tōkyō-to Ōta-ku Kamiikedai',
       'area_Ōsaka-fu Higashiōsaka-shi Aramotokita',
       'area_Ōsaka-fu Neyagawa-shi Honmachi',
       'area_Ōsaka-fu Sakai-shi Minamikawaramachi',
       'area_Ōsaka-fu Suita-shi Izumichō',
       'area_Ōsaka-fu Ōsaka-shi Fuminosato',
       'area_Ōsaka-fu Ōsaka-shi Kyōmachibori',
       'area_Ōsaka-fu Ōsaka-shi Kyūtarōmachi',
       'area_Ōsaka-fu Ōsaka-shi Nakanochō',
       'area_Ōsaka-fu Ōsaka-shi Nanbasennichimae',
       'area_Ōsaka-fu Ōsaka-shi Shinmachi', 'area_Ōsaka-fu Ōsaka-shi Ōgimachi',
       'area_Ōsaka-fu Ōsaka-shi Ōhiraki',
                 
        'closed_flag', 'reserve_visitors_air', 'reserve_visitors_hpg'
        
        ]
df_to_reshape = full[selected_cols]
df_to_reshape.replace(np.nan, 0, inplace = True)# replace Missing value with 0

/Users/qingfeng/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [65]:
# transform numeric value to float type..because float type is a must for NN??
df_to_reshape['holiday_flg'] = df_to_reshape.holiday_flg.astype('float64')
#df_to_reshape['air_genre_name_ix'] = df_to_reshape.air_genre_name_ix.astype('float64')
#df_to_reshape['air_area_name_ix'] = df_to_reshape.air_area_name_ix.astype('float64')
df_to_reshape['day'] = df_to_reshape.day.astype('float64')
df_to_reshape['month'] = df_to_reshape.month.astype('float64')
df_to_reshape['year'] = df_to_reshape.year.astype('float64')
df_to_reshape['dayofweek'] = df_to_reshape.dayofweek.astype('float64')
df_to_reshape['dayofyear'] = df_to_reshape.dayofyear.astype('float64')
df_to_reshape['day_after_holiday_flg'] = df_to_reshape.day_after_holiday_flg.astype('float64')
df_to_reshape['day_before_holiday_flg'] = df_to_reshape.day_before_holiday_flg.astype('float64')
df_to_reshape['week_of_month'] = df_to_reshape.week_of_month.astype('float64')
df_to_reshape['open_days'] = df_to_reshape.open_days.astype('float64')
df_to_reshape['closed_flag'] = df_to_reshape.closed_flag.astype('float64')

df_to_reshape.dtypes

/Users/qingfeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/qingfeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/qingfeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

air_store_id                                                  object
calendar_date                                         datetime64[ns]
visitors                                                     float64
holiday_flg                                                  float64
latitude                                                     float64
longitude                                                    float64
day                                                          float64
month                                                        float64
year                                                         float64
dayofweek                                                    float64
dayofyear                                                    float64
lookback_visitors_avg_store                                  float64
lookback_visitors_avg_genre                                  float64
lookback_visitors_avg_area                                   float64
avg_temperature                   

### step2

In [66]:
from sklearn.preprocessing import MinMaxScaler
columns_list_toscale = selected_cols[3:] 
# we do not scale 'visitors' 

In [67]:
# bound all numerical values between -1 and 1
# note: to avoid data leakage 'fit' should be made on traid data and 'transform' on train and test data
# in this case all data in test set is taken from train set, thus fit/transform on all data 
df_scaled = df_to_reshape.copy()
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_scaled[columns_list_toscale])
df_scaled[columns_list_toscale] = scaler.transform(df_scaled[columns_list_toscale])
df_scaled.sort_values(by=['air_store_id','calendar_date'], inplace = True)

#scaler.inverse_transform(scaled_data) # for inverse transformation

In [68]:
# we use log(1+x) to transform visitors
df_scaled['visitors'] = np.log1p(df_scaled['visitors'])

#Y = np.log1p(Y)
#back = np.expm1(Y)

In [69]:
df_scaled.head()

,air_store_id,calendar_date,visitors,holiday_flg,latitude,longitude,day,month,year,dayofweek,dayofyear,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area,genre_Asian,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_International cuisine,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_Fukuoka-ken Fukuoka-shi Daimyō,area_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_Fukuoka-ken Fukuoka-shi Imaizumi,area_Fukuoka-ken Fukuoka-shi Momochi,area_Fukuoka-ken Fukuoka-shi Shiobaru,area_Fukuoka-ken Fukuoka-shi Takatori,area_Fukuoka-ken Fukuoka-shi Tenjin,area_Fukuoka-ken Fukuoka-shi Torikai,area_Fukuoka-ken Itoshima-shi Maebarunishi,area_Fukuoka-ken Kitakyūshū-shi Konyamachi,area_Fukuoka-ken Kitakyūshū-shi None,area_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_Fukuoka-ken Kurume-shi Jōnanmachi,area_Fukuoka-ken Yame-shi Motomachi,area_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_Hokkaidō Asahikawa-shi 6 Jōdōri,area_Hokkaidō Katō-gun Motomachi,area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_Hyōgo-ken Himeji-shi Yasuda,area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,area_Hyōgo-ken Kōbe-shi Kumoidōri,area_Hyōgo-ken Kōbe-shi Motomachidōri,area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,area_Hyōgo-ken Nishinomiya-shi Rokutanjichō,area_Hyōgo-ken Takarazuka-shi Tōyōchō,area_Miyagi-ken Sendai-shi Kamisugi,area_Niigata-ken Kashiwazaki-shi Chūōchō,area_Niigata-ken Nagaoka-shi Ōtedōri,area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,area_Niigata-ken Niigata-shi Teraohigashi,area_Shizuoka-ken Hamamatsu-shi Motoshirochō,area_Shizuoka-ken Numazu-shi Miyukichō,area_Shizuoka-ken Shizuoka-shi Ōtemachi,area_Tōkyō-to Adachi-ku Chūōhonchō,area_Tōkyō-to Bunkyō-ku Kasuga,area_Tōkyō-to Bunkyō-ku Sendagi,area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō,area_Tōkyō-to Chiyoda-ku Kudanminami,area_Tōkyō-to Chūō-ku Ginza,area_Tōkyō-to Chūō-ku Tsukiji,area_Tōkyō-to Edogawa-ku Chūō,area_Tōkyō-to Fuchū-shi Miyanishichō,area_Tōkyō-to Itabashi-ku Itabashi,area_Tōkyō-to Katsushika-ku Tateishi,area_Tōkyō-to Kita-ku Ōjihonchō,area_Tōkyō-to Koganei-shi Honchō,area_Tōkyō-to Kōtō-ku Kameido,area_Tōkyō-to Kōtō-ku Tomioka,area_Tōkyō-to Kōtō-ku Tōyō,area_Tōkyō-to Machida-shi Morino,area_Tōkyō-to Meguro-ku Jiyūgaoka,area_Tōkyō-to Meguro-ku Kamimeguro,area_Tōkyō-to Meguro-ku Takaban,area_Tōkyō-to Minato-ku Akasaka,area_Tōkyō-to Minato-ku Roppongi,area_Tōkyō-to Minato-ku Shibakōen,area_Tōkyō-to Musashino-shi Kichijōji Honchō,area_Tōkyō-to Musashino-shi Midorichō,area_Tōkyō-to Nakano-ku Nakano,area_Tōkyō-to Nerima-ku Toyotamakita,area_Tōkyō-to Setagaya-ku Kitazawa,area_Tōkyō-to Setagaya-ku Okusawa,area_Tōkyō-to Setagaya-ku Sangenjaya,area_Tōkyō-to Setagaya-ku Setagaya,area_Tōkyō-to Shibuya-ku Dōgenzaka,area_Tōkyō-to Shibuya-ku Higashi,area_Tōkyō-to Shibuya-ku Jingūmae,area_Tōkyō-to Shibuya-ku Shibuya,area_Tōkyō-to Shinagawa-ku Higashigotanda,area_Tōkyō-to Shinagawa-ku Hiromachi,area_Tōkyō-to Shinjuku-ku Kabukichō,area_Tōkyō-to Shinjuku-ku Nishishinjuku,area_Tōkyō-to Suginami-ku Asagayaminami,area_Tōkyō-to Tachikawa-shi Izumichō,area_Tōkyō-to Taitō-ku Asakusa,area_Tōkyō-to Taitō-ku Higashiueno,area_Tōkyō-to Taitō-ku Kuramae,area_Tōkyō-to Toshima-ku Higashiikebukuro,area_Tōkyō-to Toshima-ku Mejiro,area_Tōkyō-to Toshima-ku Minamiikebukuro,area_Tō

In [70]:
df_scaled.tail()

,air_store_id,calendar_date,visitors,holiday_flg,latitude,longitude,day,month,year,dayofweek,dayofyear,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area,genre_Asian,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_International cuisine,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_Fukuoka-ken Fukuoka-shi Daimyō,area_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_Fukuoka-ken Fukuoka-shi Imaizumi,area_Fukuoka-ken Fukuoka-shi Momochi,area_Fukuoka-ken Fukuoka-shi Shiobaru,area_Fukuoka-ken Fukuoka-shi Takatori,area_Fukuoka-ken Fukuoka-shi Tenjin,area_Fukuoka-ken Fukuoka-shi Torikai,area_Fukuoka-ken Itoshima-shi Maebarunishi,area_Fukuoka-ken Kitakyūshū-shi Konyamachi,area_Fukuoka-ken Kitakyūshū-shi None,area_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_Fukuoka-ken Kurume-shi Jōnanmachi,area_Fukuoka-ken Yame-shi Motomachi,area_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_Hokkaidō Asahikawa-shi 6 Jōdōri,area_Hokkaidō Katō-gun Motomachi,area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_Hyōgo-ken Himeji-shi Yasuda,area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,area_Hyōgo-ken Kōbe-shi Kumoidōri,area_Hyōgo-ken Kōbe-shi Motomachidōri,area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,area_Hyōgo-ken Nishinomiya-shi Rokutanjichō,area_Hyōgo-ken Takarazuka-shi Tōyōchō,area_Miyagi-ken Sendai-shi Kamisugi,area_Niigata-ken Kashiwazaki-shi Chūōchō,area_Niigata-ken Nagaoka-shi Ōtedōri,area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,area_Niigata-ken Niigata-shi Teraohigashi,area_Shizuoka-ken Hamamatsu-shi Motoshirochō,area_Shizuoka-ken Numazu-shi Miyukichō,area_Shizuoka-ken Shizuoka-shi Ōtemachi,area_Tōkyō-to Adachi-ku Chūōhonchō,area_Tōkyō-to Bunkyō-ku Kasuga,area_Tōkyō-to Bunkyō-ku Sendagi,area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō,area_Tōkyō-to Chiyoda-ku Kudanminami,area_Tōkyō-to Chūō-ku Ginza,area_Tōkyō-to Chūō-ku Tsukiji,area_Tōkyō-to Edogawa-ku Chūō,area_Tōkyō-to Fuchū-shi Miyanishichō,area_Tōkyō-to Itabashi-ku Itabashi,area_Tōkyō-to Katsushika-ku Tateishi,area_Tōkyō-to Kita-ku Ōjihonchō,area_Tōkyō-to Koganei-shi Honchō,area_Tōkyō-to Kōtō-ku Kameido,area_Tōkyō-to Kōtō-ku Tomioka,area_Tōkyō-to Kōtō-ku Tōyō,area_Tōkyō-to Machida-shi Morino,area_Tōkyō-to Meguro-ku Jiyūgaoka,area_Tōkyō-to Meguro-ku Kamimeguro,area_Tōkyō-to Meguro-ku Takaban,area_Tōkyō-to Minato-ku Akasaka,area_Tōkyō-to Minato-ku Roppongi,area_Tōkyō-to Minato-ku Shibakōen,area_Tōkyō-to Musashino-shi Kichijōji Honchō,area_Tōkyō-to Musashino-shi Midorichō,area_Tōkyō-to Nakano-ku Nakano,area_Tōkyō-to Nerima-ku Toyotamakita,area_Tōkyō-to Setagaya-ku Kitazawa,area_Tōkyō-to Setagaya-ku Okusawa,area_Tōkyō-to Setagaya-ku Sangenjaya,area_Tōkyō-to Setagaya-ku Setagaya,area_Tōkyō-to Shibuya-ku Dōgenzaka,area_Tōkyō-to Shibuya-ku Higashi,area_Tōkyō-to Shibuya-ku Jingūmae,area_Tōkyō-to Shibuya-ku Shibuya,area_Tōkyō-to Shinagawa-ku Higashigotanda,area_Tōkyō-to Shinagawa-ku Hiromachi,area_Tōkyō-to Shinjuku-ku Kabukichō,area_Tōkyō-to Shinjuku-ku Nishishinjuku,area_Tōkyō-to Suginami-ku Asagayaminami,area_Tōkyō-to Tachikawa-shi Izumichō,area_Tōkyō-to Taitō-ku Asakusa,area_Tōkyō-to Taitō-ku Higashiueno,area_Tōkyō-to Taitō-ku Kuramae,area_Tōkyō-to Toshima-ku Higashiikebukuro,area_Tōkyō-to Toshima-ku Mejiro,area_Tōkyō-to Toshima-ku Minamiikebukuro,area_Tō

# Traning and Validation Input Data Format Preparation

LSTM cell accept input in shpae(n_samples, t_timesteps, m_features), therefore we need to prep the data into this format. Also we need to create our own data generator for model fitting, because 829 samples with the so many features won't fit in the memory once..

1. training / validation / testing dataframe split
2. create generator function to yield encoder_input_x, decoder_input_x, 

### step1 

In [71]:
df_train = df_scaled[(df_scaled['calendar_date'] >= train_enc_start) & (df_scaled['calendar_date'] <= train_pred_end)]
df_validation = df_scaled[(df_scaled['calendar_date'] >= val_enc_start) & (df_scaled['calendar_date'] <= val_pred_end)]
df_test = df_scaled[(df_scaled['calendar_date'] >= test_enc_start) & (df_scaled['calendar_date'] <= test_pred_end)]

In [72]:
df_train.head()

,air_store_id,calendar_date,visitors,holiday_flg,latitude,longitude,day,month,year,dayofweek,dayofyear,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area,genre_Asian,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_International cuisine,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_Fukuoka-ken Fukuoka-shi Daimyō,area_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_Fukuoka-ken Fukuoka-shi Imaizumi,area_Fukuoka-ken Fukuoka-shi Momochi,area_Fukuoka-ken Fukuoka-shi Shiobaru,area_Fukuoka-ken Fukuoka-shi Takatori,area_Fukuoka-ken Fukuoka-shi Tenjin,area_Fukuoka-ken Fukuoka-shi Torikai,area_Fukuoka-ken Itoshima-shi Maebarunishi,area_Fukuoka-ken Kitakyūshū-shi Konyamachi,area_Fukuoka-ken Kitakyūshū-shi None,area_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_Fukuoka-ken Kurume-shi Jōnanmachi,area_Fukuoka-ken Yame-shi Motomachi,area_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_Hokkaidō Asahikawa-shi 6 Jōdōri,area_Hokkaidō Katō-gun Motomachi,area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_Hyōgo-ken Himeji-shi Yasuda,area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,area_Hyōgo-ken Kōbe-shi Kumoidōri,area_Hyōgo-ken Kōbe-shi Motomachidōri,area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,area_Hyōgo-ken Nishinomiya-shi Rokutanjichō,area_Hyōgo-ken Takarazuka-shi Tōyōchō,area_Miyagi-ken Sendai-shi Kamisugi,area_Niigata-ken Kashiwazaki-shi Chūōchō,area_Niigata-ken Nagaoka-shi Ōtedōri,area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,area_Niigata-ken Niigata-shi Teraohigashi,area_Shizuoka-ken Hamamatsu-shi Motoshirochō,area_Shizuoka-ken Numazu-shi Miyukichō,area_Shizuoka-ken Shizuoka-shi Ōtemachi,area_Tōkyō-to Adachi-ku Chūōhonchō,area_Tōkyō-to Bunkyō-ku Kasuga,area_Tōkyō-to Bunkyō-ku Sendagi,area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō,area_Tōkyō-to Chiyoda-ku Kudanminami,area_Tōkyō-to Chūō-ku Ginza,area_Tōkyō-to Chūō-ku Tsukiji,area_Tōkyō-to Edogawa-ku Chūō,area_Tōkyō-to Fuchū-shi Miyanishichō,area_Tōkyō-to Itabashi-ku Itabashi,area_Tōkyō-to Katsushika-ku Tateishi,area_Tōkyō-to Kita-ku Ōjihonchō,area_Tōkyō-to Koganei-shi Honchō,area_Tōkyō-to Kōtō-ku Kameido,area_Tōkyō-to Kōtō-ku Tomioka,area_Tōkyō-to Kōtō-ku Tōyō,area_Tōkyō-to Machida-shi Morino,area_Tōkyō-to Meguro-ku Jiyūgaoka,area_Tōkyō-to Meguro-ku Kamimeguro,area_Tōkyō-to Meguro-ku Takaban,area_Tōkyō-to Minato-ku Akasaka,area_Tōkyō-to Minato-ku Roppongi,area_Tōkyō-to Minato-ku Shibakōen,area_Tōkyō-to Musashino-shi Kichijōji Honchō,area_Tōkyō-to Musashino-shi Midorichō,area_Tōkyō-to Nakano-ku Nakano,area_Tōkyō-to Nerima-ku Toyotamakita,area_Tōkyō-to Setagaya-ku Kitazawa,area_Tōkyō-to Setagaya-ku Okusawa,area_Tōkyō-to Setagaya-ku Sangenjaya,area_Tōkyō-to Setagaya-ku Setagaya,area_Tōkyō-to Shibuya-ku Dōgenzaka,area_Tōkyō-to Shibuya-ku Higashi,area_Tōkyō-to Shibuya-ku Jingūmae,area_Tōkyō-to Shibuya-ku Shibuya,area_Tōkyō-to Shinagawa-ku Higashigotanda,area_Tōkyō-to Shinagawa-ku Hiromachi,area_Tōkyō-to Shinjuku-ku Kabukichō,area_Tōkyō-to Shinjuku-ku Nishishinjuku,area_Tōkyō-to Suginami-ku Asagayaminami,area_Tōkyō-to Tachikawa-shi Izumichō,area_Tōkyō-to Taitō-ku Asakusa,area_Tōkyō-to Taitō-ku Higashiueno,area_Tōkyō-to Taitō-ku Kuramae,area_Tōkyō-to Toshima-ku Higashiikebukuro,area_Tōkyō-to Toshima-ku Mejiro,area_Tōkyō-to Toshima-ku Minamiikebukuro,area_Tō

In [73]:
df_train.tail()

,air_store_id,calendar_date,visitors,holiday_flg,latitude,longitude,day,month,year,dayofweek,dayofyear,lookback_visitors_avg_store,lookback_visitors_avg_genre,lookback_visitors_avg_area,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,avg_wind_speed,avg_humidity,day_after_holiday_flg,day_before_holiday_flg,week_of_month,open_days,lookback_visitors_std_store,lookback_visitors_std_genre,lookback_visitors_std_area,genre_Asian,genre_Bar/Cocktail,genre_Cafe/Sweets,genre_Creative cuisine,genre_Dining bar,genre_International cuisine,genre_Italian/French,genre_Izakaya,genre_Japanese food,genre_Karaoke/Party,genre_Okonomiyaki/Monja/Teppanyaki,genre_Other,genre_Western food,genre_Yakiniku/Korean food,area_Fukuoka-ken Fukuoka-shi Daimyō,area_Fukuoka-ken Fukuoka-shi Hakata Ekimae,area_Fukuoka-ken Fukuoka-shi Imaizumi,area_Fukuoka-ken Fukuoka-shi Momochi,area_Fukuoka-ken Fukuoka-shi Shiobaru,area_Fukuoka-ken Fukuoka-shi Takatori,area_Fukuoka-ken Fukuoka-shi Tenjin,area_Fukuoka-ken Fukuoka-shi Torikai,area_Fukuoka-ken Itoshima-shi Maebarunishi,area_Fukuoka-ken Kitakyūshū-shi Konyamachi,area_Fukuoka-ken Kitakyūshū-shi None,area_Fukuoka-ken Kitakyūshū-shi Ōtemachi,area_Fukuoka-ken Kurume-shi Jōnanmachi,area_Fukuoka-ken Yame-shi Motomachi,area_Hiroshima-ken Fukuyama-shi Higashisakuramachi,area_Hiroshima-ken Hiroshima-shi Kokutaijimachi,area_Hiroshima-ken Hiroshima-shi Nagarekawachō,area_Hokkaidō Abashiri-shi Minami 6 Jōhigashi,area_Hokkaidō Asahikawa-shi 6 Jōdōri,area_Hokkaidō Katō-gun Motomachi,area_Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō,area_Hokkaidō Sapporo-shi Kita 24 Jōnishi,area_Hokkaidō Sapporo-shi Kotoni 2 Jō,area_Hokkaidō Sapporo-shi Minami 3 Jōnishi,area_Hyōgo-ken Amagasaki-shi Higashinanamatsuchō,area_Hyōgo-ken Himeji-shi Yasuda,area_Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,area_Hyōgo-ken Kōbe-shi Kumoidōri,area_Hyōgo-ken Kōbe-shi Motomachidōri,area_Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,area_Hyōgo-ken Nishinomiya-shi Rokutanjichō,area_Hyōgo-ken Takarazuka-shi Tōyōchō,area_Miyagi-ken Sendai-shi Kamisugi,area_Niigata-ken Kashiwazaki-shi Chūōchō,area_Niigata-ken Nagaoka-shi Ōtedōri,area_Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,area_Niigata-ken Niigata-shi Teraohigashi,area_Shizuoka-ken Hamamatsu-shi Motoshirochō,area_Shizuoka-ken Numazu-shi Miyukichō,area_Shizuoka-ken Shizuoka-shi Ōtemachi,area_Tōkyō-to Adachi-ku Chūōhonchō,area_Tōkyō-to Bunkyō-ku Kasuga,area_Tōkyō-to Bunkyō-ku Sendagi,area_Tōkyō-to Chiyoda-ku Kanda Jinbōchō,area_Tōkyō-to Chiyoda-ku Kudanminami,area_Tōkyō-to Chūō-ku Ginza,area_Tōkyō-to Chūō-ku Tsukiji,area_Tōkyō-to Edogawa-ku Chūō,area_Tōkyō-to Fuchū-shi Miyanishichō,area_Tōkyō-to Itabashi-ku Itabashi,area_Tōkyō-to Katsushika-ku Tateishi,area_Tōkyō-to Kita-ku Ōjihonchō,area_Tōkyō-to Koganei-shi Honchō,area_Tōkyō-to Kōtō-ku Kameido,area_Tōkyō-to Kōtō-ku Tomioka,area_Tōkyō-to Kōtō-ku Tōyō,area_Tōkyō-to Machida-shi Morino,area_Tōkyō-to Meguro-ku Jiyūgaoka,area_Tōkyō-to Meguro-ku Kamimeguro,area_Tōkyō-to Meguro-ku Takaban,area_Tōkyō-to Minato-ku Akasaka,area_Tōkyō-to Minato-ku Roppongi,area_Tōkyō-to Minato-ku Shibakōen,area_Tōkyō-to Musashino-shi Kichijōji Honchō,area_Tōkyō-to Musashino-shi Midorichō,area_Tōkyō-to Nakano-ku Nakano,area_Tōkyō-to Nerima-ku Toyotamakita,area_Tōkyō-to Setagaya-ku Kitazawa,area_Tōkyō-to Setagaya-ku Okusawa,area_Tōkyō-to Setagaya-ku Sangenjaya,area_Tōkyō-to Setagaya-ku Setagaya,area_Tōkyō-to Shibuya-ku Dōgenzaka,area_Tōkyō-to Shibuya-ku Higashi,area_Tōkyō-to Shibuya-ku Jingūmae,area_Tōkyō-to Shibuya-ku Shibuya,area_Tōkyō-to Shinagawa-ku Higashigotanda,area_Tōkyō-to Shinagawa-ku Hiromachi,area_Tōkyō-to Shinjuku-ku Kabukichō,area_Tōkyō-to Shinjuku-ku Nishishinjuku,area_Tōkyō-to Suginami-ku Asagayaminami,area_Tōkyō-to Tachikawa-shi Izumichō,area_Tōkyō-to Taitō-ku Asakusa,area_Tōkyō-to Taitō-ku Higashiueno,area_Tōkyō-to Taitō-ku Kuramae,area_Tōkyō-to Toshima-ku Higashiikebukuro,area_Tōkyō-to Toshima-ku Mejiro,area_Tōkyō-to Toshima-ku Minamiikebukuro,area_Tō

### step2

In [74]:
# Tensorflow only accept numpy array as input so we need to convert the format 
# and also reshpe input data(all the numeric features) and target output(visitors) it into(n_sampels, t_timesteps, m_features)
def df_to_array(df_data): #
    y_index = df_data.columns.get_loc("visitors")
    #Y_array = df_data.values[:,y_index]\
    #                 .reshape(df_data.air_store_id.nunique(), df_data.calendar_date.nunique(), 1)
    #X_array = df_data.values[:,(y_index + 1):]\
    #                 .reshape(df_data.air_store_id.nunique(), df_data.calendar_date.nunique(), df_data.shape[1]-3)
    
    #return X_array, Y_array
    X_y_array = df_data.values[:,y_index:]\
                     .reshape(df_data.air_store_id.nunique(), df_data.calendar_date.nunique(), -1)
    # the first elements in 3rd dimension is visitors - target variable, and are following with other features
    return X_y_array

In [75]:
train_array = df_to_array(df_train)
validation_array = df_to_array(df_validation)
test_array = df_to_array(df_test)

In [76]:
NONSTATIONARY_INDEX = df_scaled.iloc[:,2:].columns.get_loc("closed_flag") # all the features following closed flag is nonstationary
VISITORS_INDEX = df_scaled.iloc[:,2:].columns.get_loc("visitors") # target value

In [86]:
NONSTATIONARY_INDEX

143

In [79]:
# create a random data generator, each iteration it will generator
# encoding input (batch_size stores, 400 days, m_features )
# decoding input (batch_size stores, 39 days, # of stationary geatures)
# decoder target (batch_size stores, 39 days, 1)
def enc_dec_data_generator(data, batch_size): 
    while 1:
        n_samples = data.shape[0]
        if batch_size <= n_samples:
            # randomly select batchsize samples out of data
            batch_samples = random.sample(range(0, n_samples), batch_size) 
            selected_data = data[batch_samples]
            
            # we need to delete those selected samples, so that we won't sample duplicates
            mask = np.ones(n_samples, bool) 
            mask[batch_samples]=False
            data = data[mask]
        else:
            # if the remaining samples are less than the batch_size we will select all the remaining data
            selected_data = data 
        
        # now Let's tease out the selected data into X_enc_in, X_dec_in, Y_dec_out
        #print(selected_data.shape)
        X_enc_in = selected_data[:, :LOOKBACK_DAYS, :] # we keep all features including visitors number in lookback periods, 
        X_dec_in = selected_data[:, LOOKBACK_DAYS:, VISITORS_INDEX + 1 : NONSTATIONARY_INDEX] # only keep stationary features in prediction periods
        Y_dec_out = selected_data[:, LOOKBACK_DAYS:, VISITORS_INDEX].reshape(-1, PRED_DAYS, 1) # only keep visitors as target value in prediction periods
        #print(Y_dec_out.shape)
        #print(X_enc_in.shape)
        #print(X_dec_in)
        yield ([X_enc_in, X_dec_in], Y_dec_out)
        #.reshape(n_samples, PRED_DAYS,1)
              

In [371]:
# test if this self defined generaotr works
#next(train_generator)

(2, 39, 1)
(2, 401, 14)
[[[-1.0 0.0769230769230771 -0.5098039215686274 -0.7033525094612756
   -0.36214692771867973 -0.8 -0.8181818181818182 1.0 0.6666666666666665
   -0.8136986301369863]
  [-1.0 0.0769230769230771 -0.5098039215686274 -0.7033525094612756
   -0.36214692771867973 -0.7333333333333334 -0.8181818181818182 1.0 1.0
   -0.8082191780821918]
  [-1.0 0.0769230769230771 -0.5098039215686274 -0.7033525094612756
   -0.36214692771867973 -0.6666666666666666 -0.8181818181818182 1.0 -1.0
   -0.8027397260273972]
  [-1.0 0.0769230769230771 -0.5098039215686274 -0.7033525094612756
   -0.36214692771867973 -0.6 -0.8181818181818182 1.0 -0.6666666666666667
   -0.7972602739726027]
  [-1.0 0.0769230769230771 -0.5098039215686274 -0.7033525094612756
   -0.36214692771867973 -0.5333333333333333 -0.8181818181818182 1.0
   -0.33333333333333337 -0.7917808219178082]
  [-1.0 0.0769230769230771 -0.5098039215686274 -0.7033525094612756
   -0.36214692771867973 -0.4666666666666667 -0.8181818181818182 1.0 0.0
   

([array([[[0.0, 1.0, 0.0769230769230771, ..., 1.0, -1.0, -1.0],
          [0.0, 1.0, 0.0769230769230771, ..., 1.0, -1.0, -1.0],
          [0.0, 1.0, 0.0769230769230771, ..., 1.0, -1.0, -1.0],
          ...,
          [1.3862943611198906, -1.0, 0.0769230769230771, ..., -1.0,
           -0.998215082552432, -1.0],
          [4.07753744390572, -1.0, 0.0769230769230771, ..., -1.0,
           -0.9526996876394467, -1.0],
          [4.0943445622221, -1.0, 0.0769230769230771, ..., -1.0,
           -0.9625167336010709, -1.0]],
  
         [[0.0, 1.0, -0.6923076923076923, ..., 1.0, -1.0, -1.0],
          [0.0, 1.0, -0.6923076923076923, ..., 1.0, -1.0, -1.0],
          [0.0, 1.0, -0.6923076923076923, ..., 1.0, -1.0, -1.0],
          ...,
          [3.1354942159291497, -1.0, -0.6923076923076923, ..., -1.0, -1.0,
           -1.0],
          [3.6109179126442243, -1.0, -0.6923076923076923, ..., -1.0, -1.0,
           -1.0],
          [3.4339872044851463, -1.0, -0.6923076923076923, ..., -1.0, -1.0,
   

# Build Model

Model Structure:

We will use past 400 days data to predict visitors traffic of following 39 days
The encoder would consume information including visitors, reservation info, weather info, holiday info
Decoder would also be fed with future 39 days data with stationary/certain information eg. holiday info, day of week etc.  to predict visitors as output

We hope that the model can pick up latent pattern based on past 400 days data and with the aids of future 39 days info, the boy can predict decent outcome for us. So let's get do it.

Model code referenced from [https://towardsdatascience.com/time-series-forecasting-with-deep-stacked-unidirectional-and-bidirectional-lstms-de7c099bd918]

In [94]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, LSTMCell, RNN, Bidirectional, concatenate
from keras.optimizers import Adam
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [99]:
def create_model(layers, n_enc_in_features, n_dec_in_features, n_dec_out_features, activation = 'tanh', dropout_rate = 0.5, optimizer = 'adam', bidirectional=False):
    
    n_layers = len(layers)
    
    ## Encoder
    encoder_inputs = Input(shape=(None, n_enc_in_features))
    lstm_cells = [LSTMCell(hidden_dim, activation = activation, dropout = dropout_rate) for hidden_dim in layers]
    if bidirectional:
        encoder = Bidirectional(RNN(lstm_cells, return_state=True))
        encoder_outputs_and_states = encoder(encoder_inputs)
        bi_encoder_states = encoder_outputs_and_states[1:]
        encoder_states = []
        for i in range(int(len(bi_encoder_states)/2)):
            temp = concatenate([bi_encoder_states[i],bi_encoder_states[2*n_layers + i]], axis=-1)
            encoder_states.append(temp)
    else:  
        encoder = RNN(lstm_cells, return_state=True, name = 'encoder')
        encoder_outputs_and_states = encoder(encoder_inputs)
        encoder_states = encoder_outputs_and_states[1:]
    
    ## Decoder
    decoder_inputs = Input(shape=(None, n_dec_in_features))
    
    if bidirectional:
        decoder_cells = [LSTMCell(hidden_dim*2) for hidden_dim in layers]
    else:
        decoder_cells = [LSTMCell(hidden_dim, activation = activation, dropout = dropout_rate) for hidden_dim in layers]
        
    decoder_lstm = RNN(decoder_cells, return_sequences=True, return_state=True, name = 'decoder')
    decoder_outputs_and_states = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_outputs = decoder_outputs_and_states[0]
    decoder_dense1 = Dense(100, activation = 'linear')
    decoder_outputs = decoder_dense1(decoder_outputs)
    decoder_dense2 = TimeDistributed(Dense(1, activation = 'linear'))
    #decoder_dense2 = Dense(n_dec_out_features, activation = 'linear') 
    decoder_outputs = decoder_dense2(decoder_outputs)
    
    model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
    return model


In [100]:
df_train.columns.shape
df_train.columns

Index(['air_store_id', 'calendar_date', 'visitors', 'holiday_flg', 'latitude',
       'longitude', 'day', 'month', 'year', 'dayofweek', 'dayofyear',
       'lookback_visitors_avg_store', 'lookback_visitors_avg_genre',
       'lookback_visitors_avg_area', 'avg_temperature', 'high_temperature',
       'low_temperature', 'precipitation', 'hours_sunlight', 'avg_wind_speed',
       'avg_humidity', 'day_after_holiday_flg', 'day_before_holiday_flg',
       'week_of_month', 'open_days', 'lookback_visitors_std_store',
       'lookback_visitors_std_genre', 'lookback_visitors_std_area',
       'genre_Asian', 'genre_Bar/Cocktail', 'genre_Cafe/Sweets',
       'genre_Creative cuisine', 'genre_Dining bar',
       'genre_International cuisine', 'genre_Italian/French', 'genre_Izakaya',
       'genre_Japanese food', 'genre_Karaoke/Party',
       'genre_Okonomiyaki/Monja/Teppanyaki', 'genre_Other',
       'genre_Western food', 'genre_Yakiniku/Korean food',
       'area_Fukuoka-ken Fukuoka-shi Daimyō',
  

In [101]:
N_enc_in_features = 146
N_dec_in_features = 142
N_dec_out_features = 1

In [102]:
#We pass the hidden dimensions in each layer in the form of a list. 
#so length of the list becomes depth of the network.  
#model = create_model([60,60], bidirectional = False)  

#Let's start from simple 1 layer structure, with 256 hidden layer
model = create_model([128], N_enc_in_features, N_dec_in_features, N_dec_out_features, bidirectional = False) 

In [103]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 146)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, 142)    0                                            
__________________________________________________________________________________________________
encoder (RNN)                   [(None, 128), (None, 140800      input_2[0][0]                    
__________________________________________________________________________________________________
decoder (RNN)                   [(None, None, 128),  138752      input_3[0][0]                    
                                                                 encoder[0][1]              

In [104]:
BATCH_SIZE = 256 # each time we train on only 256 stores 439 days records
N_TRIANING_INSTANCE = 829
STEP_SIZE=N_TRIANING_INSTANCE//BATCH_SIZE # number of training instances // each batch size
EPOCH = 10

# Hyperparameters Tunning

1. Optimizers
2. Model Structure, we will try more neurons and deeper layers
3. activation function
4. dropout rate

In [166]:
import keras
learning_rates = [1e-5, 1e-4,1e-3]
optimizers = ['adam', 'rmsprop', 'adagrad']
for o in optimizers:
    for i in learning_rates:
        print('----------optimizers-', o)
        print('learning rate: ', i)
        model = create_model([256], N_enc_in_features, N_dec_in_features, N_dec_out_features, bidirectional = False) 

        if o == 'adam':
            optimize_instance = keras.optimizers.Adam(learning_rate = i, beta_1=0.9, beta_2=0.999, amsgrad=False)
        elif o == 'rmsprop':
            optimize_instance = keras.optimizers.RMSprop(learning_rate= i, rho=0.9, decay = 1e-6)
        elif o == 'adagrad':
            optimize_instance = keras.optimizers.Adagrad(learning_rate= i)

        model.compile(optimize_instance, loss = 'mean_squared_error')
        train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
        validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
        model.fit_generator(generator = train_generator,
                            steps_per_epoch= STEP_SIZE, 
                            # Total number of steps (batches of samples) to yield from generator 
                            # before declaring one epoch finished and starting the next epoch.
                            validation_data=validation_generator,
                            validation_steps = STEP_SIZE,
                            epochs=3)

----------optimizers- adam
learning rate:  1e-05
Epoch 1/3
12/12 [==============================] - 17s 1s/step - loss: 6.2646 - val_loss: 6.4511
Epoch 2/3
12/12 [==============================] - 15s 1s/step - loss: 5.4914 - val_loss: 6.2749
Epoch 3/3
12/12 [==============================] - 14s 1s/step - loss: 4.9904 - val_loss: 5.7669
----------optimizers- adam
learning rate:  0.0001
Epoch 1/3
12/12 [==============================] - 16s 1s/step - loss: 4.8954 - val_loss: 2.9125
Epoch 2/3
12/12 [==============================] - 14s 1s/step - loss: 1.6655 - val_loss: 1.4934
Epoch 3/3
12/12 [==============================] - 14s 1s/step - loss: 1.4918 - val_loss: 1.4698
----------optimizers- adam
learning rate:  0.001
Epoch 1/3
12/12 [==============================] - 16s 1s/step - loss: 2.0787 - val_loss: 1.4008
Epoch 2/3
12/12 [==============================] - 14s 1s/step - loss: 1.3095 - val_loss: 1.5171
Epoch 3/3
12/12 [==============================] - 13s 1s/step - loss: 1.225

optimizer candidate:
1. adagrad with 0.001 learning rate
2. rmsprop with 0.0001 learning rate
3. adam with 0.001 learning rate

Then we test these 3 comparable learning rate on the 3 model structure.

## After we decided on the optimizer and learning rate, Let's try on other 

In [110]:
layers = [[256],[512],[1024],[128,128],[256,256],[512,512],[512,256],[1024,1024]]
activations = ['relu', 'tanh', 'sigmoid']
optimizers = ['adagrad','rmsprop','adam']
dropoutrates = [0.5, 0.4, 0.3, 0.2]

In [108]:
param_grid = dict(
            activation = activations, 
            optimizer = optimizers, 
            dropout_rate = dropoutrates,
            layers = layers
)
import itertools as it
param_combinations = [dict(zip(param_grid,v)) for v in it.product(*param_grid.values())]

In [115]:
param_combinations[:2]

[{'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.5},
 {'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.4}]

In [116]:
d_a = {'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.5}
d_b = {'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.4}

In [117]:
rows = []
rows.append(d_a)
rows.append(d_b)
rows

[{'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.5},
 {'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.4}]

In [118]:
pd.DataFrame.from_dict(rows, orient='columns')

,activation,dropout_rate,optimizer
0,relu,0.5,adagrad
1,relu,0.4,adagrad


In [109]:
# Now Let's do a simulated GridSearch and keep track of the result in a dataframe
rows = [] 
for i in param_combination:
    o = param_combinations[i]['optimizer']
    model = create_model(**i, N_enc_in_features, N_dec_in_features, N_dec_out_features, bidirectional = False) 

    if o == 'adam':
        optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    elif o == 'rmsprop':
        optimize_instance = keras.optimizers.RMSprop(learning_rate= 0.0001, rho=0.9, decay = 1e-6)
    elif o == 'adagrad':
        optimize_instance = keras.optimizers.Adagrad(learning_rate= 0.001)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    history = model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=EPOCH)
    
    i['loss'] = history.history['loss']
    i['loss_last'] = history.history['loss'][-1]
    i['loss_std'] = numpy.array(history.history['loss']).std()
    
    i['val_loss'] = history.history['val_loss']
    i['val_loss_last'] = history.history['val_loss'][-1]
    i['val_loss_std'] = numpy.array(history.history['val_loss']).std()
    
    rows.append(i)

GridSearch_result = pd.DataFrame.from_dict(rows, orient='columns')

[{'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.5},
 {'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.4},
 {'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.3},
 {'activation': 'relu', 'optimizer': 'adagrad', 'dropout_rate': 0.2},
 {'activation': 'relu', 'optimizer': 'rmsprop', 'dropout_rate': 0.5},
 {'activation': 'relu', 'optimizer': 'rmsprop', 'dropout_rate': 0.4},
 {'activation': 'relu', 'optimizer': 'rmsprop', 'dropout_rate': 0.3},
 {'activation': 'relu', 'optimizer': 'rmsprop', 'dropout_rate': 0.2},
 {'activation': 'relu', 'optimizer': 'adam', 'dropout_rate': 0.5},
 {'activation': 'relu', 'optimizer': 'adam', 'dropout_rate': 0.4},
 {'activation': 'relu', 'optimizer': 'adam', 'dropout_rate': 0.3},
 {'activation': 'relu', 'optimizer': 'adam', 'dropout_rate': 0.2},
 {'activation': 'tanh', 'optimizer': 'adagrad', 'dropout_rate': 0.5},
 {'activation': 'tanh', 'optimizer': 'adagrad', 'dropout_rate': 0.4},
 {'activation': 'tanh', 'optimiz

## Try Different Model Structure

In [229]:
hidden_neurons = [[128],[256],[512],[768],[1024]]
for i in hidden_neurons:
    print('----------hidden nurons-----', i)
    model = create_model(i, N_enc_in_features, N_dec_in_features, N_dec_out_features, bidirectional = False) 
    
    optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        shuffle = False,
                        epochs=3)

----------hidden nurons----- [128]
Epoch 1/3
12/12 [==============================] - 9s 791ms/step - loss: 3.2033 - val_loss: 1.5937
Epoch 2/3
12/12 [==============================] - 6s 475ms/step - loss: 1.8816 - val_loss: 1.5297
Epoch 3/3
12/12 [==============================] - 6s 470ms/step - loss: 1.7348 - val_loss: 1.4416
----------hidden nurons----- [256]
Epoch 1/3
12/12 [==============================] - 17s 1s/step - loss: 2.8102 - val_loss: 1.6192
Epoch 2/3
12/12 [==============================] - 14s 1s/step - loss: 1.5596 - val_loss: 1.5320
Epoch 3/3
12/12 [==============================] - 14s 1s/step - loss: 1.4174 - val_loss: 1.4250
----------hidden nurons----- [512]
Epoch 1/3
12/12 [==============================] - 39s 3s/step - loss: 2.7947 - val_loss: 1.5016
Epoch 2/3
12/12 [==============================] - 37s 3s/step - loss: 1.7284 - val_loss: 1.3991
Epoch 3/3
12/12 [==============================] - 37s 3s/step - loss: 1.5151 - val_loss: 1.3622
----------hidden

256 and 512 nueron performs better, Let's try deeper layer

In [230]:
hidden_neurons = [[256,256],[512,512],[512,256],[256,256,256],[512,512,512]]
for i in hidden_neurons:
    print('----------hidden nurons-----', i)
    model = create_model(i, N_enc_in_features, N_dec_in_features, N_dec_out_features, bidirectional = False) 
    
    optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=3)

----------hidden nurons----- [256, 256]
Epoch 1/3
12/12 [==============================] - 36s 3s/step - loss: 2.5559 - val_loss: 2.0347
Epoch 2/3
12/12 [==============================] - 34s 3s/step - loss: 1.6611 - val_loss: 1.4511
Epoch 3/3
12/12 [==============================] - 36s 3s/step - loss: 1.5173 - val_loss: 1.3936
----------hidden nurons----- [512, 512]
Epoch 1/3
12/12 [==============================] - 95s 8s/step - loss: 2.3860 - val_loss: 1.5688
Epoch 2/3
12/12 [==============================] - 95s 8s/step - loss: 1.6379 - val_loss: 1.3936
Epoch 3/3
12/12 [==============================] - 98s 8s/step - loss: 1.5729 - val_loss: 1.4656
----------hidden nurons----- [512, 256]
Epoch 1/3
12/12 [==============================] - 72s 6s/step - loss: 2.7590 - val_loss: 1.4820
Epoch 2/3
12/12 [==============================] - 68s 6s/step - loss: 1.6605 - val_loss: 1.6177
Epoch 3/3
12/12 [==============================] - 67s 6s/step - loss: 1.5463 - val_loss: 1.5468
-------

In [231]:
hidden_neurons = [[256,256],[512,512],[512,256],[128,128,128]]
for i in hidden_neurons:
    print('----------hidden nurons-----', i)
    model = create_model(i, N_enc_in_features, N_dec_in_features, N_dec_out_features, bidirectional = False) 
    
    optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=5)



----------hidden nurons----- [256, 256]
Epoch 1/5
12/12 [==============================] - 38s 3s/step - loss: 2.5481 - val_loss: 1.6022
Epoch 2/5
12/12 [==============================] - 35s 3s/step - loss: 1.6896 - val_loss: 1.4863
Epoch 3/5
12/12 [==============================] - 37s 3s/step - loss: 1.5602 - val_loss: 1.5460
Epoch 4/5
12/12 [==============================] - 36s 3s/step - loss: 1.5389 - val_loss: 1.6264
Epoch 5/5
12/12 [==============================] - 36s 3s/step - loss: 1.4848 - val_loss: 1.9315
----------hidden nurons----- [512, 512]
Epoch 1/5
12/12 [==============================] - 100s 8s/step - loss: 2.5320 - val_loss: 1.7141
Epoch 2/5
12/12 [==============================] - 94s 8s/step - loss: 1.6432 - val_loss: 1.7421
Epoch 3/5
12/12 [==============================] - 93s 8s/step - loss: 1.5537 - val_loss: 1.7336
Epoch 4/5
12/12 [==============================] - 92s 8s/step - loss: 1.5067 - val_loss: 1.9798
Epoch 5/5
12/12 [=============================

candidate:
[512], [256,256], [512,512], [128,128,128]

## Try Different Optimizer

In [235]:
hidden_neurons = [[512], [256,256], [512,512], [128,128,128]]
optimizers = ['adam','rmsprop', 'adagrad']
for i in hidden_neurons:
    for o in optimizers:
        print('----------optimizers-', o)
        print('----------hidden nurons-----', i)
        model = create_model(i, N_enc_in_features, N_dec_in_features, N_dec_out_features, bidirectional = False) 

        if o == 'adam':
            optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        elif o == 'rmsprop':
            optimize_instance = keras.optimizers.RMSprop(learning_rate= 0.0001, rho=0.9, decay = 1e-6)
        elif o == 'adagrad':
            optimize_instance = keras.optimizers.Adagrad(learning_rate= 0.001)

        model.compile(optimize_instance, loss = 'mean_squared_error')
        train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
        validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
        model.fit_generator(generator = train_generator,
                            steps_per_epoch= STEP_SIZE, 
                            # Total number of steps (batches of samples) to yield from generator 
                            # before declaring one epoch finished and starting the next epoch.
                            validation_data=validation_generator,
                            validation_steps = STEP_SIZE,
                            epochs=3)

----------optimizers- adam
----------hidden nurons----- [512]
Epoch 1/3
12/12 [==============================] - 43s 4s/step - loss: 2.6021 - val_loss: 1.5779
Epoch 2/3
12/12 [==============================] - 41s 3s/step - loss: 1.7511 - val_loss: 1.5793
Epoch 3/3
12/12 [==============================] - 40s 3s/step - loss: 1.6367 - val_loss: 1.6258
----------optimizers- rmsprop
----------hidden nurons----- [512]
Epoch 1/3
12/12 [==============================] - 42s 3s/step - loss: 2.8269 - val_loss: 1.5059
Epoch 2/3
12/12 [==============================] - 41s 3s/step - loss: 1.8170 - val_loss: 1.5370
Epoch 3/3
12/12 [==============================] - 41s 3s/step - loss: 1.7344 - val_loss: 1.5257
----------optimizers- adagrad
----------hidden nurons----- [512]
Epoch 1/3
12/12 [==============================] - 42s 3s/step - loss: 2.2190 - val_loss: 1.5542
Epoch 2/3
12/12 [==============================] - 38s 3s/step - loss: 1.8305 - val_loss: 1.8760
Epoch 3/3
12/12 [===============

Best Pick: [512] or [256] neurons and use Adam as potimizer

## Try Different Activation

In [237]:
activations = ['tanh','sigmoid','relu','selu']
for i in activations:
    print('----------activations-', i)
    model = create_model([512,512], N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = i, bidirectional = False) 

    optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=5)

----------activations- tanh
Epoch 1/5
12/12 [==============================] - 102s 9s/step - loss: 2.3668 - val_loss: 1.4802
Epoch 2/5
12/12 [==============================] - 91s 8s/step - loss: 1.7370 - val_loss: 1.5994
Epoch 3/5
12/12 [==============================] - 91s 8s/step - loss: 1.6743 - val_loss: 1.6149
Epoch 4/5
12/12 [==============================] - 91s 8s/step - loss: 1.6383 - val_loss: 1.6113
Epoch 5/5
12/12 [==============================] - 92s 8s/step - loss: 1.5987 - val_loss: 1.7307
----------activations- sigmoid
Epoch 1/5
12/12 [==============================] - 95s 8s/step - loss: 4.9904 - val_loss: 2.1028
Epoch 2/5
12/12 [==============================] - 91s 8s/step - loss: 1.7968 - val_loss: 1.7378
Epoch 3/5
12/12 [==============================] - 91s 8s/step - loss: 1.5336 - val_loss: 1.5814
Epoch 4/5
12/12 [==============================] - 90s 8s/step - loss: 1.4604 - val_loss: 1.5093
Epoch 5/5
12/12 [==============================] - 90s 8s/step - lo

KeyboardInterrupt: 

In [239]:
activations = ['tanh','sigmoid']
hidden_neurons = [[512],[256]]
for i in hidden_neurons:
    for a in activations:
        print('----------activation-', a)
        print('----------hidden nurons-----', i)
        model = create_model(i, N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = a, bidirectional = False) 

        optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

        model.compile(optimize_instance, loss = 'mean_squared_error')
        train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
        validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
        model.fit_generator(generator = train_generator,
                            steps_per_epoch= STEP_SIZE, 
                            # Total number of steps (batches of samples) to yield from generator 
                            # before declaring one epoch finished and starting the next epoch.
                            validation_data=validation_generator,
                            validation_steps = STEP_SIZE,
                            epochs=5)

----------activation- tanh
----------hidden nurons----- [512]
Epoch 1/5
12/12 [==============================] - 39s 3s/step - loss: 2.8773 - val_loss: 1.8106
Epoch 2/5
12/12 [==============================] - 38s 3s/step - loss: 2.0157 - val_loss: 1.5469
Epoch 3/5
12/12 [==============================] - 38s 3s/step - loss: 1.7891 - val_loss: 1.6028
Epoch 4/5
12/12 [==============================] - 38s 3s/step - loss: 1.6799 - val_loss: 1.4993
Epoch 5/5
12/12 [==============================] - 38s 3s/step - loss: 1.5783 - val_loss: 1.7017
----------activation- sigmoid
----------hidden nurons----- [512]
Epoch 1/5
12/12 [==============================] - 40s 3s/step - loss: 3.5647 - val_loss: 2.0985
Epoch 2/5
12/12 [==============================] - 38s 3s/step - loss: 1.7251 - val_loss: 1.7030
Epoch 3/5
12/12 [==============================] - 39s 3s/step - loss: 1.5676 - val_loss: 1.5096
Epoch 4/5
12/12 [==============================] - 39s 3s/step - loss: 1.4903 - val_loss: 1.5239


## try different dropout rate

In [242]:
dropoutrates = [0.1, 0.2, 0.4]
for i in dropoutrates:
    print('----------dropoutrate-', i)
    model = create_model([512], N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = 'sigmoid', dropout_rate = i, bidirectional = False) 

    optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=5)

----------activations- 0.1
Epoch 1/5
12/12 [==============================] - 41s 3s/step - loss: 3.8233 - val_loss: 1.6427
Epoch 2/5
12/12 [==============================] - 46s 4s/step - loss: 1.8970 - val_loss: 1.7215
Epoch 3/5
12/12 [==============================] - 44s 4s/step - loss: 1.6343 - val_loss: 1.6022
Epoch 4/5
12/12 [==============================] - 43s 4s/step - loss: 1.5874 - val_loss: 1.5163
Epoch 5/5
12/12 [==============================] - 41s 3s/step - loss: 1.5524 - val_loss: 1.5101
----------activations- 0.2
Epoch 1/5
12/12 [==============================] - 47s 4s/step - loss: 2.5026 - val_loss: 1.3752
Epoch 2/5
12/12 [==============================] - 41s 3s/step - loss: 1.5830 - val_loss: 1.4344
Epoch 3/5
12/12 [==============================] - 42s 4s/step - loss: 1.4827 - val_loss: 1.4080
Epoch 4/5
12/12 [==============================] - 46s 4s/step - loss: 1.4457 - val_loss: 1.4019
Epoch 5/5
12/12 [==============================] - 47s 4s/step - loss: 1.

dropout rate 0.4

## Test Other Model

In [241]:
hidden_neurons = [[512,512],[256,256]]
for i in hidden_neurons:
    print('----------hidden neurons-', i)
    model = create_model(i, N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = 'sigmoid', dropout_rate = 0.5, bidirectional = False) 

    optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

    model.compile(optimize_instance, loss = 'mean_squared_error')
    train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
    validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
    model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=5)

----------hidden neurons- [512, 512]
Epoch 1/5
12/12 [==============================] - 111s 9s/step - loss: 5.9563 - val_loss: 2.5186
Epoch 2/5
12/12 [==============================] - 101s 8s/step - loss: 2.2479 - val_loss: 2.0188
Epoch 3/5
12/12 [==============================] - 103s 9s/step - loss: 1.8204 - val_loss: 1.6211
Epoch 4/5
12/12 [==============================] - 104s 9s/step - loss: 1.7071 - val_loss: 1.6015
Epoch 5/5
12/12 [==============================] - 106s 9s/step - loss: 1.6761 - val_loss: 1.5407
----------hidden neurons- [256, 256]
Epoch 1/5
12/12 [==============================] - 46s 4s/step - loss: 3.9038 - val_loss: 2.7703
Epoch 2/5
12/12 [==============================] - 42s 4s/step - loss: 2.0918 - val_loss: 1.5866
Epoch 3/5
12/12 [==============================] - 39s 3s/step - loss: 1.9149 - val_loss: 1.6411
Epoch 4/5
12/12 [==============================] - 39s 3s/step - loss: 1.8756 - val_loss: 1.6647
Epoch 5/5
12/12 [==============================]

## More searching 
try differ


In [119]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
# patience = 3 means that if the accuracy on test set does not improve over 3 epochs we will stop the training

In [244]:
model = create_model([512], N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = 'sigmoid', dropout_rate = 0.4, bidirectional = False) 

optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimize_instance, loss = 'mean_squared_error')
train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
model.fit_generator(generator = train_generator,
                    steps_per_epoch= STEP_SIZE, 
                    # Total number of steps (batches of samples) to yield from generator 
                    # before declaring one epoch finished and starting the next epoch.
                    validation_data=validation_generator,
                    validation_steps = STEP_SIZE,
                    epochs=30,
                    callbacks=[es])

Epoch 1/30
12/12 [==============================] - 46s 4s/step - loss: 3.4105 - val_loss: 2.4260
Epoch 2/30
12/12 [==============================] - 41s 3s/step - loss: 1.8990 - val_loss: 1.3250
Epoch 3/30
12/12 [==============================] - 39s 3s/step - loss: 1.6711 - val_loss: 1.4890
Epoch 4/30
12/12 [==============================] - 38s 3s/step - loss: 1.5823 - val_loss: 1.4830
Epoch 5/30
12/12 [==============================] - 38s 3s/step - loss: 1.5662 - val_loss: 1.4194
Epoch 00005: early stopping


In [255]:
model = create_model([512], N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = 'sigmoid', dropout_rate = 0.4, bidirectional = False) 

optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimize_instance, loss = 'mean_squared_error')

BATCH_SIZE = 256 # each time we train on only 256 stores 439 days records
N_TRIANING_INSTANCE = 829
STEP_SIZE=N_TRIANING_INSTANCE//BATCH_SIZE # number of training instances // each batch size

train_generator = enc_dec_data_generator(train_array, BATCH_SIZE) # change the batch_size
validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
model.fit_generator(generator = train_generator,
                    steps_per_epoch= STEP_SIZE, 
                    # Total number of steps (batches of samples) to yield from generator 
                    # before declaring one epoch finished and starting the next epoch.
                    validation_data=validation_generator,
                    validation_steps = STEP_SIZE,
                    epochs=30,
                    callbacks=[es])

Epoch 1/30
3/3 [==============================] - 29s 10s/step - loss: 7.5578 - val_loss: 3.0277
Epoch 2/30
3/3 [==============================] - 10s 3s/step - loss: 2.1994 - val_loss: 3.3240
Epoch 3/30
3/3 [==============================] - 10s 3s/step - loss: 2.9572 - val_loss: 1.8553
Epoch 4/30
3/3 [==============================] - 10s 3s/step - loss: 1.5591 - val_loss: 2.3604
Epoch 5/30
3/3 [==============================] - 10s 3s/step - loss: 2.0832 - val_loss: 2.0928
Epoch 6/30
3/3 [==============================] - 10s 3s/step - loss: 1.5477 - val_loss: 1.6878
Epoch 7/30
3/3 [==============================] - 10s 3s/step - loss: 1.5863 - val_loss: 1.9065
Epoch 8/30
3/3 [==============================] - 10s 3s/step - loss: 1.6181 - val_loss: 1.6488
Epoch 9/30
3/3 [==============================] - 10s 3s/step - loss: 1.4086 - val_loss: 1.7866
Epoch 10/30
3/3 [==============================] - 10s 3s/step - loss: 1.5173 - val_loss: 1.7581
Epoch 11/30
3/3 [=====================

In [258]:
model = create_model([512], N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = 'tanh', dropout_rate = 0.4, bidirectional = False) 

optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimize_instance, loss = 'mean_squared_error')

BATCH_SIZE = 256 # each time we train on only 256 stores 439 days records
N_TRIANING_INSTANCE = 829
STEP_SIZE=N_TRIANING_INSTANCE//BATCH_SIZE # number of training instances // each batch size

train_generator = enc_dec_data_generator(train_array, BATCH_SIZE) # change the batch_size
validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
model.fit_generator(generator = train_generator,
                    steps_per_epoch= STEP_SIZE, 
                    # Total number of steps (batches of samples) to yield from generator 
                    # before declaring one epoch finished and starting the next epoch.
                    validation_data=validation_generator,
                    validation_steps = STEP_SIZE,
                    epochs=30,
                    callbacks=[es])

Epoch 1/30
3/3 [==============================] - 27s 9s/step - loss: 4.6676 - val_loss: 1.6872
Epoch 2/30
3/3 [==============================] - 10s 3s/step - loss: 2.0381 - val_loss: 2.2853
Epoch 3/30
3/3 [==============================] - 10s 3s/step - loss: 2.2345 - val_loss: 1.7344
Epoch 4/30
3/3 [==============================] - 10s 3s/step - loss: 1.7896 - val_loss: 1.5765
Epoch 5/30
3/3 [==============================] - 10s 3s/step - loss: 1.8173 - val_loss: 1.5540
Epoch 6/30
3/3 [==============================] - 11s 4s/step - loss: 1.5534 - val_loss: 1.7141
Epoch 7/30
3/3 [==============================] - 11s 4s/step - loss: 1.6828 - val_loss: 1.6544
Epoch 8/30
3/3 [==============================] - 10s 3s/step - loss: 1.6260 - val_loss: 1.5291
Epoch 9/30
3/3 [==============================] - 10s 3s/step - loss: 1.6516 - val_loss: 1.4985
Epoch 10/30
3/3 [==============================] - 10s 3s/step - loss: 1.5408 - val_loss: 1.5476
Epoch 11/30
3/3 [======================

In [257]:
model = create_model([256,256], N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = 'tanh', dropout_rate = 0.4, bidirectional = False) 

optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimize_instance, loss = 'mean_squared_error')
train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
model.fit_generator(generator = train_generator,
                    steps_per_epoch= STEP_SIZE, 
                    # Total number of steps (batches of samples) to yield from generator 
                    # before declaring one epoch finished and starting the next epoch.
                    validation_data=validation_generator,
                    validation_steps = STEP_SIZE,
                    epochs=30,
                    callbacks=[es])

Epoch 1/30
3/3 [==============================] - 35s 12s/step - loss: 4.3480 - val_loss: 1.7123
Epoch 2/30
3/3 [==============================] - 10s 3s/step - loss: 1.8979 - val_loss: 2.1172
Epoch 3/30
3/3 [==============================] - 11s 4s/step - loss: 2.0529 - val_loss: 1.4802
Epoch 4/30
3/3 [==============================] - 11s 4s/step - loss: 1.7136 - val_loss: 1.6784
Epoch 5/30
3/3 [==============================] - 11s 4s/step - loss: 1.6935 - val_loss: 1.4875
Epoch 6/30
3/3 [==============================] - 10s 3s/step - loss: 1.6172 - val_loss: 1.5937
Epoch 7/30
3/3 [==============================] - 10s 3s/step - loss: 1.6239 - val_loss: 1.4606
Epoch 8/30
3/3 [==============================] - 10s 3s/step - loss: 1.5753 - val_loss: 1.4786
Epoch 9/30
3/3 [==============================] - 11s 4s/step - loss: 1.6168 - val_loss: 1.4446
Epoch 10/30
3/3 [==============================] - 10s 3s/step - loss: 1.5356 - val_loss: 1.4557
Epoch 11/30
3/3 [=====================

In [246]:
model = create_model([256,256], N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = 'sigmoid', dropout_rate = 0, bidirectional = False) 

optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimize_instance, loss = 'mean_squared_error')
train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)
model.fit_generator(generator = train_generator,
                    steps_per_epoch= STEP_SIZE, 
                    # Total number of steps (batches of samples) to yield from generator 
                    # before declaring one epoch finished and starting the next epoch.
                    validation_data=validation_generator,
                    validation_steps = STEP_SIZE,
                    epochs=30,
                    callbacks=[es])

Epoch 1/30
12/12 [==============================] - 43s 4s/step - loss: 2.0407 - val_loss: 1.5797
Epoch 2/30
12/12 [==============================] - 39s 3s/step - loss: 1.8134 - val_loss: 1.5406
Epoch 3/30
12/12 [==============================] - 42s 4s/step - loss: 1.7074 - val_loss: 1.6736
Epoch 4/30
12/12 [==============================] - 39s 3s/step - loss: 1.6687 - val_loss: 1.7326
Epoch 5/30
12/12 [==============================] - 43s 4s/step - loss: 1.6588 - val_loss: 1.7095
Epoch 00005: early stopping


In [247]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 8)
# patience = 3 means that if the accuracy on test set does not improve over 10 epochs we will stop the training

In [251]:
model = create_model([256,256], N_enc_in_features, N_dec_in_features, N_dec_out_features, activation = 'tanh', dropout_rate = 0.4, bidirectional = False) 

optimize_instance = keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimize_instance, loss = 'mean_squared_error')
train_generator = enc_dec_data_generator(train_array, 256)
validation_generator = enc_dec_data_generator(validation_array, 256)
model.fit_generator(generator = train_generator,
                    steps_per_epoch= STEP_SIZE, 
                    # Total number of steps (batches of samples) to yield from generator 
                    # before declaring one epoch finished and starting the next epoch.
                    validation_data=validation_generator,
                    validation_steps = STEP_SIZE,
                    epochs=30,
                    callbacks=[es])

Epoch 1/30
12/12 [==============================] - 51s 4s/step - loss: 2.6967 - val_loss: 1.8358
Epoch 2/30
12/12 [==============================] - 37s 3s/step - loss: 1.7091 - val_loss: 1.7603
Epoch 3/30
12/12 [==============================] - 37s 3s/step - loss: 1.6133 - val_loss: 1.7830
Epoch 4/30
12/12 [==============================] - 39s 3s/step - loss: 1.5451 - val_loss: 1.9244
Epoch 5/30
12/12 [==============================] - 40s 3s/step - loss: 1.4468 - val_loss: 2.1216
Epoch 6/30
12/12 [==============================] - 39s 3s/step - loss: 1.4353 - val_loss: 2.2626
Epoch 7/30
 6/12 [==============>...............] - ETA: 16s - loss: 1.4919

KeyboardInterrupt: 

In [162]:
model.compile(Adam(), loss = 'mean_squared_error')

In [163]:
BATCH_SIZE = 64 # each time we train on only 64 stores 439 days records
N_TRIANING_INSTANCE = 829
STEP_SIZE=N_TRIANING_INSTANCE//BATCH_SIZE # number of training instances // each batch size

In [164]:
train_generator = enc_dec_data_generator(train_array, BATCH_SIZE)
validation_generator = enc_dec_data_generator(validation_array, BATCH_SIZE)

In [165]:
model.fit_generator(generator = train_generator,
                        steps_per_epoch= STEP_SIZE, 
                        # Total number of steps (batches of samples) to yield from generator 
                        # before declaring one epoch finished and starting the next epoch.
                        validation_data=validation_generator,
                        validation_steps = STEP_SIZE,
                        epochs=3)

Epoch 1/3
12/12 [==============================] - 9s 748ms/step - loss: 3.0369 - val_loss: 1.7324
Epoch 2/3
 2/12 [====>.........................] - ETA: 3s - loss: 1.5777

KeyboardInterrupt: 

## Check out the true RMSE error on validation set

In [206]:
y = validation_array[:, LOOKBACK_DAYS:, :VISITORS_INDEX + 1]
x_enc = validation_array[:, :LOOKBACK_DAYS, :]
x_dec = validation_array[:, LOOKBACK_DAYS:, VISITORS_INDEX +1: NONSTATIONARY_INDEX]

In [207]:
predicted_array = model.predict([x_enc,x_dec]).reshape(829,39)

In [208]:
target_array = y.reshape(829,39)# 829 stores in validation sets

In [212]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
# because we made prediction on log(1+visitors), so this function will gave us root mean squared log error
error = mean_squared_error(target_array, predicted_array)
print('root mean squared log error is', np.sqrt(error))

root mean squared log error is 3.111713427097839


In [213]:
# we need to transform the value back to the original scale to calculate mean square error
target_array_reversed = np.e**target_array - 1
predicted_array_reversed = np.e**predicted_array - 1
error = mean_squared_error(target_array_reversed, predicted_array_reversed)
print('root mean squared error is', np.sqrt(error))

root mean squared error is 27.05610169641363


In [223]:
target_array_reversed = np.e**target_array - 1
predicted_array_reversed = np.e**predicted_array - 1
error = mean_squared_log_error(target_array_reversed, predicted_array_reversed)
print('root mean squared error is', np.sqrt(error))

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [217]:
predicted_array_reversed[predicted_array_reversed <0 ]

array([-0.36259514, -0.35645515, -0.32560217, ..., -0.38331503,
       -0.38402897, -0.35709232], dtype=float32)

NameError: name 'log' is not defined